[Previous Notebook](token-classification-deployment.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE_RIVA_BOOTCAMP.ipynb)

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;
[1](token-classification-training.ipynb)
[2](token-classification-deployment.ipynb)
[3]
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;

# Testing a Named Entity Recognition Model in Riva - Exercise


This notebook explores taking an .riva model, the result of `tlt token_classification` command, and leveraging the Riva ServiceMaker framework to aggregate all the necessary artifacts for Riva deployment to a target environment. Once the model is deployed in Riva, you can issue inference requests to the server. . 

---
## Learning Objectives
In this notebook, you will learn how to:  
- Deploy the model(s) locally  on the Riva Server
- Send inference requests using the Riva Python API.

---
## Pre-requisites

To follow along, please make sure:
- You have access to NVIDIA NGC, and are able to download the Riva Quickstart [resources](https://ngc.nvidia.com/catalog/resources/nvidia:riva:riva_quickstart)
- Have an .riva model file that you wish to deploy. You can obtain this from `tlt <task> export` (with `export_format=RIVA`). Please refer the tutorial on *Named entity recognition using Transfer Learning Toolkit* for more details on training and exporting an .riva model.

---
## Run Inference
Once the Riva server is up and running with your models, you can send inference requests querying the server. 

To send GRPC requests, you can install Riva Python API bindings for client. This is available as a pip .whl with the QuickStart.

Otherwise, you can use the riva-client docker container which comes pre-installed with all the inference dependancies.

### Connect to Riva server and run inference
Now we actually query the Riva server, let's get started. The following cell queries the riva server(using grpc) to yield a result.

In [ ]:
%%writefile $RIVA_DIR/ner_client.py

import grpc
import os
import argparse
import riva_api.riva_nlp_pb2 as rnlp
import riva_api.riva_nlp_pb2_grpc as rnlp_srv

# use the NER network to return top-1 classes for entities
def postprocess_labels_server(tokens_response):
    results = []
    for i in range(0, len(tokens_response.results)):
        slots = []
        slot_scores = []
        tokens = []
        for j in range(0, len(tokens_response.results[i].results)):
          entity = tokens_response.results[i].results[j]
          tokens.append(entity.token)
          slots.append(entity.label[0].class_name)
          slot_scores.append(entity.label[0].score)
        results.append((slots, tokens, slot_scores))

    return results

def run_ner(grpc_server, query):
    channel = grpc.insecure_channel(grpc_server)
    riva_nlp = rnlp_srv.RivaLanguageUnderstandingStub(channel)
    req = rnlp.AnalyzeEntitiesRequest()
    req.query = query
    resp = riva_nlp.AnalyzeEntities(req)
    print("Query:", query)
    print(postprocess_labels_server(resp))

def get_args():
    parser = argparse.ArgumentParser(description="Client app to test named entity recognition on Riva")
    parser.add_argument("--server", default="localhost:50051", type=str, help="URI to GRPC server endpoint")
    parser.add_argument("--query", default="NVIDIA is located at Santa Clara", type=str, help="Input Query")
    return parser.parse_args()

def run_ner_client():
    args = get_args()
    run_ner(args.server, query=args.query)

if __name__ == '__main__':
    run_ner_client()

If you've installed the Riva Python API bindings for client using the pip whl, you can execute the above saved script with the following command:

In [ ]:
#! python3 $RIVA_DIR/ner_client.py --query "NVIDIA is located at Santa Clara" --server localhost:50051

### Running Token Classification Inference using the Riva Python API directly

We can also run Named Entity Recognition using the API directly without executing client side scripts.
An example is shown below

In [ ]:
# first import require libraries
import io
import librosa
from time import time
import numpy as np
import IPython.display as ipd
import grpc
import requests

# NLP proto
##change this configuration to match your DGX number and your RIVA port number
##for example, 'dgx0180:50055'

channel = grpc.insecure_channel('localhost:50051')

import riva_api.riva_nlp_pb2 as rnlp
import riva_api.riva_nlp_pb2_grpc as rnlp_srv


riva_nlp = rnlp_srv.RivaLanguageUnderstandingStub(channel)



# Use the TokenClassification API to run a Named Entity Recognition (NER) model
# Note: the model configuration of the NER model indicates that the labels are
# in IOB format. Riva, subsequently, knows to:
#   a) ignore 'O' labels
#   b) Remove B- and I- prefixes from labels
#   c) Collapse sequences of B- I- ... I- tokens into a single token

req = rnlp.TokenClassRequest()
req.model.model_name = "riva_ner"     # If you have deployed a custom model with the domain_name 
                                        # parameter in ServiceMaker's `riva-build` command then you should use 
                                        # "riva_ner_<your_input_domain_name>" where <your_input_domain_name>
                                        # is the name you provided to the domain_name parameter.

req.text.append("Jensen Huang is the CEO of NVIDIA Corporation, "
                "located in Santa Clara, California")
resp = riva_nlp.ClassifyTokens(req)

print("Named Entities:")
for result in resp.results[0].results:
    print(f"  {result.token} ({result.label[0].class_name})")

## Exercise :

Use the Token classification API to identify Named Entities in the following sentences
Note: you may use the client python file to compare your result against the model you choose

1: "The weather in New York today is much hotter than in San Francisco."

2: "The Sahara desert is located in Africa."

3: "The Amazon rainforest is being affected by climate change."

In [ ]:
##first, import the required libraries

import io
import librosa
from time import time
import numpy as np
import IPython.display as ipd
import grpc
import requests

# NLP proto


## next, setup the connection parameters and variables
## next, initialize the query variables




# Use the TokenClassification API to run a Named Entity Recognition (NER) model
# Note: the model configuration of the NER model indicates that the labels are
# in IOB format. Jarvis, subsequently, knows to:
#   a) ignore 'O' labels
#   b) Remove B- and I- prefixes from labels
#   c) Collapse sequences of B- I- ... I- tokens into a single token

## initialize the request object



##setup the query


##make the request




##parse the response variable and print out the entities found 


## compare results here